https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#sphx-glr-download-tutorial-10-key-features-005-visualization-py

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import torch
import pandas as pd
from pathlib import Path
import optuna
from reprpo.hp.helpers import optuna_df

In [3]:
from reprpo.training import train
from reprpo.experiments import experiment_configs
from reprpo.hp.space import search_spaces

[2024-10-05 03:11:52,176] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


## Objective

In [4]:
SEED=42
key_metric = "acc_gain_vs_ref/oos"
torch.manual_seed(SEED)
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [5]:
# silence please
import os
from loguru import logger
logger.remove()
logger.remove()
logger.add(os.sys.stderr, level="WARNING")

os.environ["WANDB_MODE"] = "disabled"
os.environ["HF_DATASETS_OFFLINE"] = "1"
os.environ["TQDM_DISABLE"] = "true"

In [6]:
f_db = f"sqlite:///optuna.db"
f = f_db.replace('sqlite:///', './')
print(f)
Path(f).parent.mkdir(parents=True, exist_ok=True)
f_db

./optuna.db


'sqlite:///optuna.db'

In [7]:
# print(f'to visualise run in cli\ncd nbs\noptuna-dashboard {f_db}')

In [8]:
from reprpo.hp.target import override, default_tuner_kwargs
from reprpo.experiments import experiment_configs
import copy
import wandb

import optuna.pruners
from optuna_integration.wandb import WeightsAndBiasesCallback

In [9]:


import functools

def list2tuples(d):
    for k, v in d.items():
        if isinstance(v, list):
            d[k] = tuple(v)
    return d

def objective_func(kwargs, trial, starter_experiment_name):
    cfg = copy.deepcopy(experiment_configs[starter_experiment_name][1])
    override(cfg, default_tuner_kwargs)
    override(cfg, kwargs)
    kwargs = list2tuples(kwargs)
    r = train(cfg, trial=trial)
    return r

def objective(trial: optuna.Trial, starter_experiment_name, trial2args, key_metric=key_metric) -> float:
    kwargs = trial2args(trial)
    r = objective_func(kwargs, trial, starter_experiment_name)
    return r[key_metric]



## Opt

Note on pruning. It's only really usefull with validation metrics and for long jobs over many epochs. I've got a small proxy job so there is no need.

In [10]:
# from reprpo.experiments import experiment_configs
from reprpo.hp.space import experiment_configs
experiment_configs.keys()

dict_keys(['hs-svd-mse', 'hs-hra-rank', 'hs-ortho-prefvec', 'ether-prefvec', 'dpo', 'projbp', 'projgrad'])

In [42]:
import warnings
warnings.filterwarnings("ignore", category=optuna.exceptions.ExperimentalWarning) 

In [43]:
from optuna.study.study import storages, get_all_study_names
study_names = get_all_study_names(storage=f_db)

for study_name in study_names:
    print(study_name)
    study = optuna.load_study(study_name=study_name, storage=f_db)
    try:
        df_res = optuna_df(study, key_metric)
        display(df_res)
        print()
    except ValueError as e:
        # print(e)
        raise
        print('-')

projgrad2


[W 2024-10-05 04:00:56,649] Study instance does not contain completed trials.


,importance,best
"projgrad2 N=0, best=nan",,



side-ether-prefvec


,importance,best
"side-ether-prefvec N=208, best=1.169",,
lr,0.368,0.000615
nb,0.219,30
β,0.218,0.403787
reduction,0.146,25
flip_side,0.014,True
use_dpo_loss,0.013,False
collect_hs,0.011,False
Htype,0.010,oft
use_nll_loss,0.000,False


...

,importance,best
projbp N=0,,



dpo


,importance,best
dpo N=0,,



hs-svd-mse


ValueError: Cannot evaluate parameter importances with only a single trial.

In [44]:
# unit test
for starter_experiment_name, trial2args in search_spaces.items():
    study = optuna.create_study(direction="maximize")
    trial = study.ask()
    print('starter_experiment_name', starter_experiment_name)
    kwargs = trial2args(trial)
    cfg = copy.deepcopy(experiment_configs[starter_experiment_name][1])
    override(cfg, default_tuner_kwargs)
    override(cfg, kwargs)
    kwargs = list2tuples(kwargs)
    print('kwargs', kwargs)

    try:
        df_res = optuna_df(study, key_metric)
        print(df_res.to_markdown())
    except Exception as e:
        print(e)

[I 2024-10-05 04:01:55,770] A new study created in memory with name: no-name-e554df91-d979-462c-8163-4fd44ef5718b
[W 2024-10-05 04:01:55,775] Study instance does not contain completed trials.
[I 2024-10-05 04:01:55,786] A new study created in memory with name: no-name-ac909c68-5aeb-495c-b3e3-8b921c43e9b8
[W 2024-10-05 04:01:55,788] Study instance does not contain completed trials.
[I 2024-10-05 04:01:55,798] A new study created in memory with name: no-name-b4cd401d-53f5-4275-9999-e1a0ac39b144
[W 2024-10-05 04:01:55,800] Study instance does not contain completed trials.
[I 2024-10-05 04:01:55,810] A new study created in memory with name: no-name-967aeaff-bda2-4443-b7a6-e7783a98f75d
[W 2024-10-05 04:01:55,813] Study instance does not contain completed trials.
[I 2024-10-05 04:01:55,824] A new study created in memory with name: no-name-9c079a73-3cbb-4d7f-a629-e69b68fbba91
[W 2024-10-05 04:01:55,826] Study instance does not contain completed trials.
[I 2024-10-05 04:01:55,833] A new study 

starter_experiment_name hs-svd-mse
kwargs {'lr': 0.0014543130814445343, 'collect_input': True, 'collect_hs': False, 'transform.quantile': 0.4, 'transform.dual_svd': True, 'loss.α': 0.4603424071642692}
| no-name-e554df91-d979-462c-8163-4fd44ef5718b N=0, best=nan   | importance   | best   |
|--------------------------------------------------------------|--------------|--------|
starter_experiment_name hs-hra-rank
kwargs {'lr': 6.639431716557299e-06, 'transform.r': 6, 'transform.apply_GS': False, 'loss.α': 0.0015318280237912813, 'loss.β': 2.749249405078123}
| no-name-ac909c68-5aeb-495c-b3e3-8b921c43e9b8 N=0, best=nan   | importance   | best   |
|--------------------------------------------------------------|--------------|--------|
starter_experiment_name hs-ortho-prefvec
kwargs {'lr': 1.8994449075034481e-06, 'transform.orthogonal_map': 'cayley', 'loss.β': 0.0016112022795036344, 'loss.use_orth_loss': False, 'loss.use_angle_loss': False, 'loss.use_dpo_loss': True, 'loss.use_nll_loss': True

In [45]:
# from optuna import trial
# t = trial.create_trial(value=1)
# t.suggest_categorical("a", [1, 2, 3])

In [46]:
MAX_TRIALS= 250
import numpy as np
spaces = list(search_spaces.items())
np.random.shuffle(spaces)

for starter_experiment_name, trial2args in search_spaces.items():

    study_name = f"{starter_experiment_name}"
    study = optuna.create_study(
        study_name=study_name,
        direction="maximize",
        load_if_exists=True,
        storage=f_db,
        sampler=optuna.samplers.TPESampler(seed=SEED),
        # pruner=optuna.pruners.NopPruner(),
    )

    n = 0
    try:
        df = study.trials_dataframe().sort_values('value', ascending=False)
        n = len(df)
    except Exception as e:
        print(e)
        pass
    if n>0:
        print(f"loaded {n} {study_name} trials")
        # print('study.best_trial', study.best_trial)

        df_res = optuna_df(study, key_metric)
        print(df_res.to_markdown())

    
    if n < MAX_TRIALS:
        _objective = functools.partial(objective, key_metric=key_metric, starter_experiment_name=starter_experiment_name, trial2args=trial2args)

        study.optimize(_objective, 
                    n_trials=MAX_TRIALS, 
                    gc_after_trial=True, 
                    catch=(AssertionError, OSError, RuntimeError, KeyError, torch.OutOfMemoryError)
        )

    print('='*80)

In [ ]:
print(wandb.run.get_url())

## plot

In [ ]:
# You can use Matplotlib instead of Plotly for visualization by simply replacing `optuna.visualization` with
# `optuna.visualization.matplotlib` in the following examples.
from optuna.visualization.matplotlib import plot_contour
from optuna.visualization.matplotlib import plot_edf
from optuna.visualization.matplotlib import plot_intermediate_values
from optuna.visualization.matplotlib import plot_optimization_history
from optuna.visualization.matplotlib import plot_parallel_coordinate
from optuna.visualization.matplotlib import plot_param_importances
from optuna.visualization.matplotlib import plot_rank
from optuna.visualization.matplotlib import plot_slice
from optuna.visualization.matplotlib import plot_timeline

In [ ]:
search_spaces.keys()

In [ ]:
starter_experiment_name = 'projgrad'
trial2args = search_spaces[starter_experiment_name]

study_name = f"{starter_experiment_name}"
study = optuna.create_study(
    study_name=study_name,
    direction="maximize",
    load_if_exists=True,
    storage=f_db,
    sampler=optuna.samplers.TPESampler(seed=SEED),
    pruner=optuna.pruners.NopPruner(),
)
print('study.best_trial', study.best_trial)
df = study.trials_dataframe().query('state == "COMPLETE"').sort_values('value', ascending=False)
print(len(df))
plot_optimization_history(study)

In [13]:
# plot_timeline(study)

In [29]:
# plot_intermediate_values(study)

In [30]:
# plot_contour(study)


In [ ]:
plot_slice(study)


In [ ]:
plot_param_importances(study)

In [ ]:
plot_intermediate_values(study)

In [ ]:
plot_parallel_coordinate(study)

### Apendix 1: dataclass 2 optuna

In [20]:
# import inspect
# import typing
# from typing import Literal

# def optuna_suggest_from_dataclass(t):
#     n = t.__name__
#     print(f'## {n}')
#     sig = inspect.signature(t)
#     for name, param in sig.parameters.items():
#         if param.annotation== bool:
#             print(f'"{name}": trial.suggest_categorical("{name}", [True, False]),')
#         elif param.annotation==int:
#             print(f'"{name}": trial.suggest_int("{name}", 1, 10),')
#         elif param.annotation ==float:
#             print(f'"{name}": trial.suggest_float("{name}", 0.1, 10.0),')
#         elif param.annotation == str:
#             print(f'"{name}": trial.suggest_categorical("{name}", ["a", "b", "c"]),')
#         elif param.annotation == tuple:
#             print(f'"{name}": trial.suggest_categorical("{name}", [(1, 2), (3, 4), (5, 6)]),')
#         elif typing.get_origin(param.annotation) == Literal:
#             print(f'"{name}": trial.suggest_categorical("{name}", {param.annotation.__args__}),')
#         else:
#             print(f"!!Unknown type {param}")
#             # print(name, param.default, param.annotation)

# optuna_suggest_from_dataclass(ReprPOConfig)
# for t in Transforms:
#     print(f'## {t}')
#     optuna_suggest_from_dataclass(t.value)
# for l in Losses:
#     print(f'## {l}')
#     optuna_suggest_from_dataclass(l.value)


# optuna_suggest_from_dataclass(DPOProjGradConfig)